# Topics and Transformation

Don't forget to set

In [2]:
import logging
import os.path

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

if you want to see logging events.

## Transformation interface

In the previous tutorial on [Corpora and Vector Spaces](https://radimrehurek.com/gensim/tut1.html), we created a corpus of documents represented as a stream of vectors. To continue, let’s fire up gensim and use that corpus:

In [3]:
from gensim import corpora, models, similarities
if (os.path.exists("simple_deerwester.dict")):
    dictionary = corpora.Dictionary.load('simple_deerwester.dict')
    corpus = corpora.MmCorpus('simple_deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

2016-11-22 16:37:12,004 : INFO : Pattern library is not installed, lemmatization won't be available.
2016-11-22 16:37:12,009 : INFO : Could not import Theano, will use standard float for default ShardedCorpus dtype.
2016-11-22 16:37:12,222 : INFO : 'pattern' package not found; tag filters are not available for English
2016-11-22 16:37:12,234 : INFO : loading Dictionary object from simple_deerwester.dict
2016-11-22 16:37:12,236 : INFO : loaded corpus index from simple_deerwester.mm.index
2016-11-22 16:37:12,238 : INFO : initializing corpus reader from simple_deerwester.mm
2016-11-22 16:37:12,249 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


Used files generated from first tutorial


In [4]:
print (dictionary[0])
print (dictionary[1])
print (dictionary[2])
type(corpus)
print corpus

interface
computer
human
MmCorpus(9 documents, 12 features, 28 non-zero entries)


In this tutorial, I will show how to transform documents from one vector representation into another. This process serves two goals:

1. To bring out hidden structure in the corpus, discover relationships between words and use them to describe the documents in a new and (hopefully) more semantic way.
1. To make the document representation more compact. This both improves efficiency (new representation consumes less resources) and efficacy (marginal data trends are ignored, noise-reduction).

### Creating a transformation

The transformations are standard Python objects, typically initialized by means of a training corpus:

In [5]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2016-11-22 16:37:12,291 : INFO : collecting document frequencies
2016-11-22 16:37:12,295 : INFO : PROGRESS: processing document #0
2016-11-22 16:37:12,299 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


We used our old corpus from tutorial 1 to initialize (train) the transformation model. Different transformations may require different initialization parameters; in case of TfIdf, the “training” consists simply of going through the supplied corpus once and computing document frequencies of all its features. Training other models, such as Latent Semantic Analysis or Latent Dirichlet Allocation, is much more involved and, consequently, takes much more time.

> <B>Note</B>:
> Transformations always convert between two specific vector spaces. The same vector space (= the same set of feature ids) must be used for training as well as for subsequent vector transformations. Failure to use the same input feature space, such as applying a different string preprocessing, using different feature ids, or using bag-of-words input vectors where TfIdf vectors are expected, will result in feature mismatch during transformation calls and consequently in either garbage output and/or runtime exceptions.

In [6]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


Or to apply a transformation to a whole corpus:

In [7]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In this particular case, we are transforming the same corpus that we used for training, but this is only incidental. Once the transformation model has been initialized, it can be used on any vectors (provided they come from the same vector space, of course), even if they were not used in the training corpus at all. This is achieved by a process called folding-in for LSA, by topic inference for LDA etc.

> <b>Note:</b> 
> Calling model[corpus] only creates a wrapper around the old corpus document stream – actual conversions are done on-the-fly, during document iteration. We cannot convert the entire corpus at the time of calling corpus_transformed = model[corpus], because that would mean storing the result in main memory, and that contradicts gensim’s objective of memory-indepedence. If you will be iterating over the transformed corpus_transformed multiple times, and the transformation is costly, serialize the resulting corpus to disk first and continue using that.

Transformations can also be serialized, one on top of another, in a sort of chain:

In [8]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2016-11-22 16:37:12,423 : INFO : using serial LSI version on this node
2016-11-22 16:37:12,429 : INFO : updating model with new documents
2016-11-22 16:37:12,447 : INFO : preparing a new chunk of documents
2016-11-22 16:37:12,449 : INFO : using 100 extra samples and 2 power iterations
2016-11-22 16:37:12,452 : INFO : 1st phase: constructing (12, 105) action matrix
2016-11-22 16:37:12,469 : INFO : orthonormalizing (12, 105) action matrix
2016-11-22 16:37:12,477 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2016-11-22 16:37:12,498 : INFO : computing the final decomposition
2016-11-22 16:37:12,500 : INFO : keeping 5 factors (discarding 11.038% of energy spectrum)
2016-11-22 16:37:12,510 : INFO : processed documents up to #9
2016-11-22 16:37:12,519 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2016-11-22 16:37:12,546 : INFO : topic #

Here we transformed our Tf-Idf corpus via [Latent Semantic Indexing](http://en.wikipedia.org/wiki/Latent_semantic_indexing) into a latent 2-D space (2-D because we set num_topics=2). Now you’re probably wondering: what do these two latent dimensions stand for? Let’s inspect with models.LsiModel.print_topics():

In [9]:
lsi.print_topics(2)

2016-11-22 16:37:12,572 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2016-11-22 16:37:12,575 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  u'-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

(the topics are printed to log – see the note at the top of this page about activating logging)

It appears that according to LSI, “trees”, “graph” and “minors” are all related words (and contribute the most to the direction of the first topic), while the second topic practically concerns itself with all the other words. As expected, the first five documents are more strongly related to the second topic while the remaining four documents to the first topic:

In [10]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

[(0, 0.066007833960904885), (1, -0.52007033063618502), (2, -0.37649581219168876), (3, 0.086159541481412166), (4, -0.75281095723843139)]
[(0, 0.19667592859142699), (1, -0.76095631677000342), (2, 0.50806745810016729), (3, -0.037796964821600852), (4, -0.096363533263836107)]
[(0, 0.089926399724466546), (1, -0.72418606267525032), (2, -0.40898973155376306), (3, -0.01518893094795401), (4, 0.27686440691934366)]
[(0, 0.07585847652178368), (1, -0.63205515860034256), (2, -0.53935336057338878), (3, 0.078275456810800392), (4, 0.34532730488012098)]
[(0, 0.10150299184980291), (1, -0.5737308483002943), (2, 0.67093385852959186), (3, -0.33929517906883178), (4, 0.069279625801279873)]
[(0, 0.70321089393783121), (1, 0.16115180214025931), (2, -0.18266089635241561), (3, -0.58259563370849832), (4, -0.046940151434534533)]
[(0, 0.87747876731198315), (1, 0.16758906864659603), (2, -0.10880822642632955), (3, -0.23079422499324298), (4, -0.0094386431367925357)]
[(0, 0.90986246868185794), (1, 0.14086553628719239), (2

In [11]:
lsi.save('simple_model.lsi') # same for tfidf, lda, ...
lsi = models.LsiModel.load('simple_model.lsi')

2016-11-22 16:37:12,646 : INFO : saving Projection object under simple_model.lsi.projection, separately None
2016-11-22 16:37:12,653 : INFO : saving LsiModel object under simple_model.lsi, separately None
2016-11-22 16:37:12,661 : INFO : not storing attribute projection
2016-11-22 16:37:12,663 : INFO : not storing attribute dispatcher
2016-11-22 16:37:12,665 : INFO : loading LsiModel object from simple_model.lsi
2016-11-22 16:37:12,668 : INFO : loading id2word recursively from simple_model.lsi.id2word.* with mmap=None
2016-11-22 16:37:12,674 : INFO : setting ignored attribute projection to None
2016-11-22 16:37:12,678 : INFO : setting ignored attribute dispatcher to None
2016-11-22 16:37:12,679 : INFO : loading LsiModel object from simple_model.lsi.projection


The next question might be: just how exactly similar are those documents to each other? Is there a way to formalize the similarity, so that for a given input document, we can order some other set of documents according to their similarity? Similarity queries are covered in the [next tutorial](https://radimrehurek.com/gensim/tut3.html).

## Available transformations

Gensim implements several popular Vector Space Model algorithms:

* [Term Frequency * Inverse Document Frequency](http://en.wikipedia.org/wiki/Tf–idf), Tf-Idf expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality, except that features which were rare in the training corpus will have their value increased. It therefore converts integer-valued vectors into real-valued ones, while leaving the number of dimensions intact. It can also optionally normalize the resulting vectors to (Euclidean) unit length.

In [12]:
model = models.TfidfModel(corpus, normalize=True)

2016-11-22 16:37:12,699 : INFO : collecting document frequencies
2016-11-22 16:37:12,702 : INFO : PROGRESS: processing document #0
2016-11-22 16:37:12,705 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


* [Latent Semantic Indexing, LSI (or sometimes LSA)](http://en.wikipedia.org/wiki/Latent_semantic_indexing) transforms documents from either bag-of-words or (preferrably) TfIdf-weighted space into a latent space of a lower dimensionality. For the toy corpus above we used only 2 latent dimensions, but on real corpora, target dimensionality of 200–500 is recommended as a “golden standard” [1].

In [13]:
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

2016-11-22 16:37:12,757 : INFO : using serial LSI version on this node
2016-11-22 16:37:12,762 : INFO : updating model with new documents
2016-11-22 16:37:12,765 : INFO : preparing a new chunk of documents
2016-11-22 16:37:12,767 : INFO : using 100 extra samples and 2 power iterations
2016-11-22 16:37:12,770 : INFO : 1st phase: constructing (12, 400) action matrix
2016-11-22 16:37:12,773 : INFO : orthonormalizing (12, 400) action matrix
2016-11-22 16:37:12,790 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2016-11-22 16:37:12,809 : INFO : computing the final decomposition
2016-11-22 16:37:12,812 : INFO : keeping 9 factors (discarding 0.000% of energy spectrum)
2016-11-22 16:37:12,816 : INFO : processed documents up to #9
2016-11-22 16:37:12,823 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2016-11-22 16:37:12,854 : INFO : topic #1

    LSI training is unique in that we can continue “training” at any point, simply by providing more training documents. This is done by incremental updates to the underlying model, in a process called online training. Because of this feature, the input document stream may even be infinite – just keep feeding LSI new documents as they arrive, while using the computed transformation model as read-only in the meanwhile!

> <b>Example</b> 
> 
> model.add_documents(another_tfidf_corpus) # now LSI has been trained on tfidf_corpus + another_tfidf_corpus
> lsi_vec = model[tfidf_vec] # convert some new document into the LSI space, without affecting the model

> model.add_documents(more_documents) # tfidf_corpus + another_tfidf_corpus + more_documents
> lsi_vec = model[tfidf_vec]


    See the [gensim.models.lsimodel](https://radimrehurek.com/gensim/models/lsimodel.html#module-gensim.models.lsimodel) documentation for details on how to make LSI gradually “forget” old observations in infinite streams. If you want to get dirty, there are also parameters you can tweak that affect speed vs. memory footprint vs. numerical precision of the LSI algorithm.

    gensim uses a novel online incremental streamed distributed training algorithm (quite a mouthful!), which I published in [5]. gensim also executes a stochastic multi-pass algorithm from Halko et al. [4] internally, to accelerate in-core part of the computations. See also 
    [Experiments on the English Wikipedia](https://radimrehurek.com/gensim/wiki.html) for further speed-ups by distributing the computation across a cluster of computers.

* [Random Projections](http://www.cis.hut.fi/ella/publications/randproj_kdd.pdf), RP aim to reduce vector space dimensionality. This is a very efficient (both memory- and CPU-friendly) approach to approximating TfIdf distances between documents, by throwing in a little randomness. Recommended target dimensionality is again in the hundreds/thousands, depending on your dataset.

In [14]:
model = models.RpModel(corpus_tfidf, num_topics=500)

2016-11-22 16:37:12,891 : INFO : no word id mapping provided; initializing from corpus, assuming identity
2016-11-22 16:37:12,896 : INFO : constructing (500, 12) random matrix


* [Latent Dirichlet Allocation, LDA](http://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).

In [15]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

2016-11-22 16:37:12,935 : INFO : using symmetric alpha at 0.01
2016-11-22 16:37:12,942 : INFO : using symmetric eta at 0.01
2016-11-22 16:37:12,943 : INFO : using serial LDA version on this node
2016-11-22 16:37:12,964 : INFO : running online LDA training, 100 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2016-11-22 16:37:12,966 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2016-11-22 16:37:13,056 : INFO : -115.920 per-word bound, 78618939038081493992847127884070912.0 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2016-11-22 16:37:13,064 : INFO : PROGRESS: pass 0, at document #9/9
2016-11-22 16:37:13,096 : INFO : topic #30 (0.010): 0.083*user + 0.083*survey + 0.083*graph + 0.083*trees + 0.083*eps + 0.083*interface + 0.083*syste

    gensim uses a fast implementation of online LDA parameter estimation based on [2], modified to run in distributed mode on a cluster of computers.

* [Hierarchical Dirichlet Process, HDP](http://jmlr.csail.mit.edu/proceedings/papers/v15/wang11a/wang11a.pdf) is a non-parametric bayesian method (note the missing number of requested topics):

In [16]:
model = models.HdpModel(corpus, id2word=dictionary)

2016-11-22 16:37:13,187 : INFO : topic 0: 0.557*survey + 0.074*eps + 0.068*trees + 0.060*human + 0.056*user + 0.055*time + 0.031*minors + 0.026*system + 0.022*response + 0.019*computer + 0.018*interface + 0.014*graph
2016-11-22 16:37:13,189 : INFO : topic 1: 0.309*eps + 0.167*computer + 0.119*survey + 0.114*minors + 0.070*human + 0.066*graph + 0.054*response + 0.040*trees + 0.022*system + 0.016*time + 0.012*interface + 0.010*user
2016-11-22 16:37:13,194 : INFO : topic 2: 0.321*trees + 0.201*interface + 0.111*human + 0.094*computer + 0.068*time + 0.054*graph + 0.040*user + 0.036*response + 0.031*system + 0.022*minors + 0.013*eps + 0.008*survey
2016-11-22 16:37:13,196 : INFO : topic 3: 0.170*eps + 0.153*trees + 0.150*user + 0.130*time + 0.103*human + 0.090*computer + 0.080*system + 0.046*graph + 0.045*interface + 0.020*minors + 0.012*survey + 0.000*response
2016-11-22 16:37:13,198 : INFO : topic 4: 0.194*system + 0.193*eps + 0.177*survey + 0.151*graph + 0.093*computer + 0.051*user + 0.04

    gensim uses a fast, online implementation based on [3]. The HDP model is a new addition to gensim, and still rough around its academic edges – use with care.

Adding new VSM transformations (such as different weighting schemes) is rather trivial; see the API reference or directly the Python code for more info and examples.

It is worth repeating that these are all unique, incremental implementations, which do not require the whole training corpus to be present in main memory all at once. With memory taken care of, I am now improving Distributed Computing, to improve CPU efficiency, too. If you feel you could contribute (by testing, providing use-cases or code), please let me know.

Continue on to the next tutorial on Similarity Queries.
